In [1]:
import pandas as pd
from sklearn import cluster
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import warnings
warnings.filterwarnings("ignore")
import seaborn as sns
from sklearn.decomposition import PCA
plt.rcParams['font.sans-serif']=['SimHei'] #图中文字体设置为黑体
plt.rcParams['axes.unicode_minus']=False
from tqdm.notebook import tqdm
from sklearn.metrics import silhouette_score
import scipy
import numpy as np
import random
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

In [2]:
df_data = pd.read_csv('舞弊案例全市场数据2014-2021.csv')
df_data

,BASIC_entity_name,BASIC_year,ASSETAB_GOODWILLQUALITY,SURPLUSAB_REVENUESTRUCTURE,COSTAB_PERIODCOST,FINANCINGAB_HIGHCASHDEBT,REVENUEAB_SELLONCREDITRATIO,RPARTYAB_VARNISHOCCUPATION,INVESTMENTAB_ASSETDISPOSAL,ASSETAB_STOCK,...,REVENUEAB_SELLINGEXPENSES,COSTAB_PROFITGROWTH,SURPLUSAB_ACCRUALS,RINVESTAB_SUBSIDIARYCASH,DILEMMAAB_OPCASH,RPARTYAB_VARNISHPROFIT,ASSETAB_LONGRECEIVABLES,INVESTMENTAB_INVESTSTRATEGY,FINANCINGAB_HIGHINTEREST,DEBTAB_FINANCINGSTRUCTURE
0,TCL科技集团股份有限公司,2014,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,TCL科技集团股份有限公司,2015,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,TCL科技集团股份有限公司,2016,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,TCL科技集团股份有限公司,2017,0,0,0,0,0,0,0,0,...,0,2,0,1,0,0,0,0,0,1
4,TCL科技集团股份有限公司,2018,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43811,龙芯中科技术股份有限公司,2017,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
43812,龙芯中科技术股份有限公司,2018,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
43813,龙芯中科技术股份有限公司,2019,0,0,2,0,0,2,0,0,...,1,2,0,2,0,2,0,0,0,1
43814,龙芯中科技术股份有限公司,2020,0,0,0,0,1,0,0,1,...,0,0,0,2,0,0,0,0,0,1


In [3]:
fraud_list = pd.read_excel('上市公司财务舞弊企业数据库 _V10.0_20230709(1).xlsx', sheet_name='财务舞弊企业数据库', skiprows=3)
fraud_list.drop([0, 1, 2], axis=0, inplace=True)
fraud_list = fraud_list[['企业名称', '涉及造假年份']]
fraud_list

,企业名称,涉及造假年份
3,福建龙净环保股份有限公司,2021；2022
4,江苏舜天股份有限公司,2009；2010；2011；2012；2013；2014；2015；2016；2017；2...
5,西陇科学股份有限公司,2020；2021；2022
6,瑞斯康达科技发展股份有限公司,2019；2020
7,浙江海盐力源环保科技股份有限公司,2021
...,...,...
291,湖北迈亚股份有限公司,2000；2001；2002；2003；2004；2005
292,成都聚友网络股份有限公司,2001；2002；2003；2004
293,丹东化学纤维股份有限公司,2003；2004；2005；2006
294,华夏建通科技开发股份有限公司,2003；2004；2005；2007


In [4]:
fraud_result = []
for _, row in fraud_list.iterrows():
    name = row['企业名称']
    if '（' in name:
        name = name[:name.index('（')]
    years = str(row['涉及造假年份']).split('；')
    for year in years:
        fraud_result.append((name, year))
fraud_result

[('福建龙净环保股份有限公司', '2021'),
 ('福建龙净环保股份有限公司', '2022'),
 ('江苏舜天股份有限公司', '2009'),
 ('江苏舜天股份有限公司', '2010'),
 ('江苏舜天股份有限公司', '2011'),
 ('江苏舜天股份有限公司', '2012'),
 ('江苏舜天股份有限公司', '2013'),
 ('江苏舜天股份有限公司', '2014'),
 ('江苏舜天股份有限公司', '2015'),
 ('江苏舜天股份有限公司', '2016'),
 ('江苏舜天股份有限公司', '2017'),
 ('江苏舜天股份有限公司', '2018'),
 ('江苏舜天股份有限公司', '2019'),
 ('江苏舜天股份有限公司', '2020'),
 ('江苏舜天股份有限公司', '2021'),
 ('西陇科学股份有限公司', '2020'),
 ('西陇科学股份有限公司', '2021'),
 ('西陇科学股份有限公司', '2022'),
 ('瑞斯康达科技发展股份有限公司', '2019'),
 ('瑞斯康达科技发展股份有限公司', '2020'),
 ('浙江海盐力源环保科技股份有限公司', '2021'),
 ('陕西航天动力高科技股份有限公司', '2016'),
 ('陕西航天动力高科技股份有限公司', '2017'),
 ('陕西航天动力高科技股份有限公司', '2018'),
 ('陕西航天动力高科技股份有限公司', '2019'),
 ('陕西航天动力高科技股份有限公司', '2020'),
 ('起步股份有限公司', '2018'),
 ('起步股份有限公司', '2019'),
 ('起步股份有限公司', '2020'),
 ('远程电缆股份有限公司', '2017'),
 ('浙江卓锦环保科技股份有限公司', '2021'),
 ('河南新宁现代物流股份有限公司', '2018'),
 ('深圳市奇信集团股份有限公司', '2015'),
 ('深圳市奇信集团股份有限公司', '2016'),
 ('深圳市奇信集团股份有限公司', '2017'),
 ('深圳市奇信集团股份有限公司', '2018'),
 ('深圳市奇信集团股份有限公司', '2020'),
 ('江苏宏图高科技股份有限公

In [5]:
fraud_rows = []
for name, year in fraud_result:
    fraud_rows.append(df_data[(df_data['BASIC_entity_name'] == name) & (df_data['BASIC_year'] == int(year))])
fraud_df = pd.concat(fraud_rows)
fraud_df

,BASIC_entity_name,BASIC_year,ASSETAB_GOODWILLQUALITY,SURPLUSAB_REVENUESTRUCTURE,COSTAB_PERIODCOST,FINANCINGAB_HIGHCASHDEBT,REVENUEAB_SELLONCREDITRATIO,RPARTYAB_VARNISHOCCUPATION,INVESTMENTAB_ASSETDISPOSAL,ASSETAB_STOCK,...,REVENUEAB_SELLINGEXPENSES,COSTAB_PROFITGROWTH,SURPLUSAB_ACCRUALS,RINVESTAB_SUBSIDIARYCASH,DILEMMAAB_OPCASH,RPARTYAB_VARNISHPROFIT,ASSETAB_LONGRECEIVABLES,INVESTMENTAB_INVESTSTRATEGY,FINANCINGAB_HIGHINTEREST,DEBTAB_FINANCINGSTRUCTURE
37561,福建龙净环保股份有限公司,2021,0,0,0,0,0,0,0,0,...,0,0,0,2,0,0,0,0,0,1
27927,江苏舜天股份有限公司,2014,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
27928,江苏舜天股份有限公司,2015,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27929,江苏舜天股份有限公司,2016,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
27930,江苏舜天股份有限公司,2017,0,0,0,0,2,0,0,0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32667,深圳市全新好股份有限公司,2014,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,1
32668,深圳市全新好股份有限公司,2015,0,1,0,0,0,1,0,0,...,0,2,0,0,0,1,0,0,0,1
5746,中科云网科技集团股份有限公司,2014,0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
10468,北讯集团股份有限公司,2014,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1


In [6]:
fraud_df.to_csv('舞弊案例数据库.csv', index=False)